In [2]:
addprocs()
#rmprocs()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
using Plots; gr()
using Rsvg
using DataFrames
using CSV
@everywhere using Distributions
@everywhere using Rotations
# @everywhere using RandomNumbers.Xorshifts
# @everywhere rng = Xoroshiro128Plus()

In [ ]:
function Q(X_Q::Array{Float64}, s_cz::Float64)
    return (1 ./ s_cz .* sqrt(2 / pi) .* (
        (3 .* X_Q.^2 ./ 2 ./ s_cz^2 .- 1) .* exp(-X_Q.^2 ./ 2 ./ s_cz^2) .-
        (4 .* X_Q.^2 ./ 3 ./ s_cz^2 .- 1) .* exp(-2 .* X_Q.^2 ./ 3 ./ s_cz^2)))
end

function R(eta::Array{Float64})
    return 3 .* eta .* (1 .- eta.^2 ./ 9) .* (1 .+ eta.^2 ./ 3).^(-5 / 2)
end

In [10]:
@everywhere const r = 1.0
@everywhere const addmatrix = [[r, 0.0, 0.0], [-r, 0.0, 0.0], [0.0, r, 0.0], [0.0, -r, 0.0], [0.0, 0.0, r], [0.0, 0.0, -r]]


@everywhere function calc_efg(el::Array{Float64})
    efg = zeros(Float64, 3, 3)
    for k in 1:size(el)[1]
        r = norm(el[k,:])
        for i in 1:3
            for j in 1:3
                efg[i, j] += -(3 * el[k,i] * el[k,j]) / r^5
                if i == j
                    efg[i, j] += 1 / r^3
                end
            end
        end
    end
    return efg
end


@everywhere function get_params(efg::Array{Float64})
    ev = eigvals(efg)
    sort_index = sortperm(abs.(ev))
    V_xx = ev[sort_index[1]]
    V_yy = ev[sort_index[2]]
    V_zz = ev[sort_index[3]]
    eta = (V_xx - V_yy) / V_zz
    dirz = eigvecs(efg)[:,sort_index[3]]
    dirx = eigvecs(efg)[:,sort_index[1]]
    return V_zz, eta, dirz, dirx
end


@everywhere function calc(fun; n::Int64=1000000, runs::Int64=1, sigma::Float64=0.01)
    Vzz = zeros(Float64, n)
    eta = zeros(Float64, n)
    dir = zeros(Float64, (n, 3))
    pos = zeros(Float64, (6, 3))
    pos = gen_okt!(pos)
    for i in 1:n
        # pos = move(pos, r, sigma)
        pos = fun(pos, runs, sigma)
        efg = calc_efg(pos)
        Vzz[i], eta[i], dir[i,:] = get_params(efg)
        #Vzz[i], eta[i], dir[i,:] = get_params(calc_efg(fun(r, sigma)))
    end

    dis = zeros(Float64, n-1)
    for i in 2:n
        dis[i-1] = sqrt(sum((dir[i-1,:] - dir[i,:]) .^ 2))
    end
    
    return Vzz, eta, dis
end

In [11]:
function rotmat_()
    theta = rand() * 2π
    phi = rand() * 2π
    z = rand() * 2.0
    r = sqrt(z)
    Vx = sin(phi) * r
    Vy = cos(phi) * r
    Vz = sqrt(2.0 - z)
    st = sin(theta)
    ct = cos(theta)
    Sx = Vx * ct - Vy * st
    Sy = Vx * st + Vy * ct
    mat = Array{Float64}(3, 3)
    mat[0,0] = Vx * Sx - ct
    mat[1,0] = Vx * Sy - st
    mat[2,0] = Vx * Sx - ct
    mat[0,1] = Vy * Sx - ct
    mat[0,1] = Vy * Sx - ct
    mat[0,1] = Vy * Sx - ct
    mat[0,0] = Vx * Sx - ct
end


@everywhere function get_frequency(theta::Float64, phi::Float64, eta::Float64, deltaqq::Float64=1.0)
    costheta::Float64 = cos(theta)
    cos2phi::Float64 = cos(2*phi)
    A::Float64 = -3.375 + 2.25 * eta * cos2phi - 0.375 * (eta * cos2phi)^2
    B::Float64 = 3.75 - 0.5 * eta^2 - 2.0 * eta * cos2phi + 0.75 * (eta * cos2phi)^2
    C::Float64 = -0.375 + 1.0 / 3.0 * eta^2 - 0.25 * eta * cos2phi - 0.375 * (eta * cos2phi)^2
    return deltaqq * (A * costheta^4 + B * costheta^2 + C)
end


@everywhere function gen_lifetime(lifetime0::Float64)
    r = rand()
    while r < 1e-20
        r = rand()
    end
    return -log(r) * lifetime0
end


@everywhere function pulvermittel_slow(n::Int64 = 100, endlife::Float64 = 32768.0, lifetime0::Float64 = 16.0, sigma = 0.01)
    times = [0.0]
    freqs = [0.0]
    for i in 1:n
        times_x, freqs_x = simulate(endlife, lifetime0, sigma)
        times = vcat(times, times_x .+ times[length(times)])
        freqs = vcat(freqs, freqs_x)
    end
    return times, freqs
end


@everywhere function pulvermittel(n::Int64 = 100, endlife::Float64 = 32768.0, lifetime0::Float64 = 16.0, sigma = 0.01)
    times = [0.0]
    freqs = [0.0]
    jobs = [@spawn simulate(endlife, lifetime0, sigma) for i=1:n]
    for j in jobs
        times_x, freqs_x = fetch(j)
        times = vcat(times, times_x .+ times[length(times)])
        freqs = vcat(freqs, freqs_x)
        #push!(freqs, freqs_x)
    end
    return times, freqs
end


@everywhere function simulate(endlife::Float64, lifetime0::Float64 = 16.0, sigma = 0.01)
    time = 0.0
    pos = gen_okt!(zeros(Float64, (6, 3)))
    rotmat = rand(RotMatrix{3})

    times::Array{Float64} = []
    freqs::Array{Float64} = []
    temp_pos = pos
    temp_freq = 0.0
    temp_Vzz = 0.0
    temp_eta = 0.0
    temp_theta = 0.0
    temp_phi = 0.0
    temp_dir = []
    while time < endlife
        pos = move!(pos, sigma=sigma)
        efg = calc_efg(pos)
        # efg = rotmat * efg
        Vzz, eta, dirz, dirx = get_params(efg)
        dirz /= norm(dirz)
        dirz = rotmat * dirz
        dirx::Array{Float64} = rotmat * dirx
        p = dirx - dot(dirx, dirz) * dirz
        theta = acos(dirz[3])
        phi = acos(dot(dirz, p) / (norm(dirz) * norm(p)))
        freq = get_frequency(theta, phi, eta, Vzz^2)
    
        
            
        push!(times, time)
        push!(freqs, freq)
        
        time += gen_lifetime(lifetime0)
    end
    return times, freqs
end

In [12]:
@everywhere function gen_okt!(pos::Array{Float64,2}, runs::Int64=1, sigma::Float64=0.01)
    randn!(pos)
    pos *= sigma
    pos[1, 1] += r
    pos[2, 1] -= r
    pos[3, 2] += r
    pos[4, 2] -= r
    pos[5, 3] += r
    pos[6, 3] -= r
    return pos
end

@everywhere function move!(pos::Array{Float64,2}; runs::Int64=1, sigma::Float64=0.01)
    #if rand() < 0.01
        # return gen_okt!(pos, runs, sigma)
    #    row = rand(1:6)
    #    pos[row,:] = sigma * randn(3) + addmatrix[row]
    #end
    for _ in 1:runs
        row = rand(1:6)
        x = sigma/20 * randn(3)
        dist = sum((pos[row,:] + x).^2)
        while (dist < 0.5) || (dist > 1.5)
            row = rand(1:6)
            x = sigma/20 * randn(3)
            dist = sqrt(sum((pos[row,:] + x).^2))
        end
        pos[row,:] += sigma/20 * randn(3) #+ addmatrix[row]
    end
    return pos    
end

In [ ]:
job1 = @spawn pulvermittel(100, 32768.0, 1.0, 0.01)
job2 = @spawn pulvermittel(100, 32768.0, 1.0, 0.001)
times_rh, freqs_rh = fetch(job1)
times_xh, freqs_xh = fetch(job2)

In [13]:
@time times_ph, freqs_ph = pulvermittel(100, 16384.0, 1.0, 0.01)
times_p, freqs_p = simulate(16384.0, 1.0, 0.01)
size(freqs_ph)

 15.197776 seconds (143.02 k allocations: 1.277 GiB, 1.07% gc time)


(1638796,)

In [ ]:
freqs_ph *= 100
freqs_p *= 100;

In [ ]:
job1 = @spawn simulate(32768.0, 1.0, 0.01)
job2 = @spawn simulate(32768.0, 1.0, 0.001)
times_r, freqs_r = fetch(job1)
times_x, freqs_x = fetch(job2)

In [15]:
#plot(times, freqs, label = "okt")
#plot(times_r, freqs_r, label = "okt r")
plot(times_p, freqs_p, label = "okt p")
#plot!(freq_iso, label = "iso")
plot!(xlims = (0, 3600))
plot!(ylims = (-0.1, 0.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 -0.10 
 
 
 -0.05 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
<polyline clip-path="url(#clip5102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,165.623 45.756,165.138 45.7672,164.325 45.8048,165.28 45.8149,164.988 46.0416,165.775 46.4084,166.825 46.7393,167.138 46.8007,167.191 47.1309,167.579 
 47.3332,167.424 47.3988,168.373 47.5469,168.612 47.593,167.362 47.6924,168.532 47.7315,169.061 48.2187,169.417 48.2258,170.323 48.7738,170.07 48.8149,169.846 
 48.9954,169.616 49.1679,168.164 49.2195,166.876 49.4859,166.429 49.5007,166.072 49.8569,165.986 49.9018,165.689 50.0219,166.31 50.1483,166.992 50.2584,167.235 
 50.485,168.086 50.5363,168.398 50.6332,167.159 50.6574,167.182 50.7715,165.394 51.0059,164.623 51.2075,165.498 51.3824,163.009 51.3964,161.275 51.7433,162.453 
 51.8583,161.212 52.4261,161.111 52.6189,159.667 52.7341,159.386 52.8184,160.202 52.8983,159.52 53.1257,158.982 53.2068,157.96 53.2161,157.616 53.2601,157.927 
 53.2849,157.482 53.4012,157.443 53.6769,156.027 53.7187,156.467 54.1123,158.825 54.6007,158.465 55.0679,156.491 55.1034,157.27 55.4547,156.951 55.5814,157.879 
 55.7437,157.885 55.8726,158.004 56.1731,156.269 56.2425,156.335 56.318,157.113 56.3288,155.916 56.3605,154.984 56.5416,153.593 56.8176,154.55 56.9637,154.826 
 57.3081,154.675 57.3228,153.666 57.3488,153.524 57.587,153.64 57.8972,154.291 58.136,153.08 58.1617,153.131 58.1621,153.233 58.305,155.618 58.3713,155.153 
 58.6596,155.547 58.8368,154.258 58.8716,154.91 58.9311,153.055 58.9318,152.976 59.0719,154.388 59.1252,154.782 59.1505,152.215 59.1851,150.456 59.1904,150.868 
 59.4263,152.004 59.5221,152.339 59.5778,151.152 60.0012,150.023 60.4577,151.298 60.4855,151.848 60.509,152.267 60.5689,152.006 60.847,152.437 60.8688,153.906 
 60.9633,153.327 61.0556,156.015 61.0581,156.483 61.2736,156.408 61.5085,157.632 61.7328,155.744 61.8389,158.138 62.5972,157.358 63.2512,156.873 63.8239,156.634 
 63.9154,157.976 63.9427,157.755 64.1054,155.397 64.1275,157.956 64.1709,158.816 64.2898,158.616 64.39,158.918 64.4247,158.459 64.6612,156.92 64.6736,156.575 
 65.4318,158.709 65.6353,158.879 65.8275,159.399 65.973,160.713 66.0481,160.956 66.2972,160.714 66.8659,161.402 67.1366,161.742 67.1416,160.111 67.1705,159.896 
 67.3336,157.729 67.5907,158.562 67.6351,157.817 67.6829,157.07 67.9894,156.068 68.0599,156.484 68.1247,157.611 68.4958,156.999 68.5137,156.762 68.618,157.733 
 68.6379,157.564 68.7584,157.03 68.7811,156.819 69.1421,156.409 69.2045,156.575 69.261,155.004 69.3967,156.373 69.4135,156.588 69.4694,157.645 69.5243,154.746 
 69.5406,155.102 69.6603,153.022 69.8728,153.229 69.8836,151.585 69.9308,152.129 70.1038,150.8 70.2197,150.363 70.2318,153.169 70.2572,153.599 70.3027,155.414 
 70.3339,155.504 70.7317,154.559 70.7838,155.929 70.9046,155.238 71.4117,154.786 71.5813,154.324 71.589,153.17 71.5931,151.98 71.7945,152.684 71.8228,152.543 
 71.8551,153.445 71.8824,152.431 71.8825,152.391 71.9722,152.021 71.9989,151.578 72.0968,151.949 72.1079,151.699 72.5857,151.126 72.8431,151.497 72.8727,150.891 
 73.1349,152.28 73.2997,152.324 73.4065,151.906 73.5578,152.125 73.8545,152.864 74.0036,150.149 74.1612,148.912 74.4784,148.714 74.5333,148.551 74.6355,148.679 
 74.7729,147.588 74.9135,147.128 75.0565,149.171 75.3207,147.479 75.6208,148.948 75.6214,148.207 76.083,148.779 76.1464,150.11 76.2024,150.415 76.3593,148.677 
 76.4879,148.477 76.5319,148.854 76.7021,147.705 76.7429,145.907 77.2303,146.534 77.2521,146.375 77.2729,145.789 77.4015,145.508 77.4371,145.463 77.761,145.662 
 77.7896,145.986 77.8637,144.727 77.8977,143.63 77.9465,146.421 78.0793,148.056 78.208,147.539 78.6585,147.216 78.6774,148.329 78.8418,150.269 78.8494,149.203 
 79.0241,149.302 79.1572,149.628 79.2499,150.548 79.2868,148.877 79.4027,148.666 79.527,147.03 79.719,147.082 79.8096,146.47 79.8684,147.75 79.9267

In [ ]:
savefig("traj vgl iso.pdf")

In [ ]:
n = length(freqs_p)
diff_p = Array{Float64}(n-1)
for i in 2:n
    diff_p[i-1] = abs(freqs_p[i] - freqs_p[i-1])
end
histogram(diff_p, label = "okt p", bins = 2500)
plot!(xlims = (0, 2))

In [ ]:
basedir = "/home/jens/Documents/simulation/NMRSimulation/test/okttest/"
freq_iso = CSV.read("$(basedir)freq_iso.txt", types=[Float64],  delim = " ")
freq_iso = convert(Array{Float64}, freq_iso)

In [14]:
#histogram(freqs_rh, label = "okt sigma0.01", bins = 500)
# freqs_ph *= 0.8
# histogram(freq_iso, normed=true, label = "iso", bins = 70)
histogram(freqs_ph, normed=true,  label = "okt diff", bins = 200)
#plot(xlims = (-0.5, 0.5))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
<polyline clip-path="url(#clip4902)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 22.3377,375.813 22.3377,375.375 27.1067,375.375 27.1067,373.102 31.8757,373.102 31.8757,373.291 36.6448,373.291 36.6448,374.548 41.4138,374.548 41.4138,372.735 
 46.1828,372.735 46.1828,367.381 50.9519,367.381 50.9519,353.939 55.7209,353.939 55.7209,347.676 60.49,347.676 60.49,331.624 65.259,331.624 65.259,309.142 
 70.028,309.142 70.028,298.157 74.7971,298.157 74.7971,257.753 79.5661,257.753 79.5661,167.663 84.3351,167.663 84.3351,11.811 89.1042,11.811 89.1042,20.6242 
 93.8732,20.6242 93.8732,64.6079 98.6422,64.6079 98.6422,73.7662 103.411,73.7662 103.411,101.463 108.18,101.463 108.18,127.686 112.949,127.686 112.949,150.42 
 117.718,150.42 117.718,184.021 122.487,184.021 122.487,209.174 127.256,209.174 127.256,228.284 132.026,228.284 132.026,244.355 136.795,244.355 136.795,254.376 
 141.564,254.376 141.564,261.609 146.333,261.609 146.333,281.845 151.102,281.845 151.102,293.929 155.871,293.929 155.871,297.01 160.64,297.01 160.64,310.334 
 165.409,310.334 165.409,320.349 170.178,320.349 170.178,323.074 174.947,323.074 174.947,325.123 179.716,325.123 179.716,331.164 184.485,331.164 184.485,334.719 
 189.254,334.719 189.254,342.264 194.023,342.264 194.023,348.752 198.792,348.752 198.792,352.277 203.561,352.277 203.561,355.038 208.33,355.038 208.33,357.423 
 213.099,357.423 213.099,359.945 217.868,359.945 217.868,360.31 222.637,360.31 222.637,360.019 227.406,360.019 227.406,361.868 232.175,361.868 232.175,364.464 
 236.944,364.464 236.944,364.097 241.713,364.097 241.713,363.434 246.482,363.434 246.482,364.886 251.251,364.886 251.251,364.483 256.02,364.483 256.02,365.502 
 260.789,365.502 260.789,364.979 265.559,364.979 265.559,366.31 270.328,366.31 270.328,368.027 275.097,368.027 275.097,369.41 279.866,369.41 279.866,370.273 
 284.635,370.273 284.635,371.086 289.404,371.086 289.404,372.045 294.173,372.045 294.173,372.932 298.942,372.932 298.942,372.639 303.711,372.639 303.711,372.836 
 308.48,372.836 308.48,372.398 313.249,372.398 313.249,371.664 318.018,371.664 318.018,372.228 322.787,372.228 322.787,371.7 327.556,371.7 327.556,371.793 
 332.325,371.793 332.325,370.695 337.094,370.695 337.094,371.489 341.863,371.489 341.863,372.552 346.632,372.552 346.632,373.13 351.401,373.13 351.401,373.025 
 356.17,373.025 356.17,373.13 360.939,373.13 360.939,373.699 365.708,373.699 365.708,373.935 370.477,373.935 370.477,374.055 375.246,374.055 375.246,373.68 
 380.015,373.68 380.015,373.458 384.784,373.458 384.784,373.058 389.553,373.058 389.553,373.236 394.323,373.236 394.323,373.455 399.092,373.455 399.092,373.554 
 403.861,373.554 403.861,374.104 408.63,374.104 408.63,374.504 413.399,374.504 413.399,374.934 418.168,374.934 418.168,375.17 422.937,375.17 422.937,374.907 
 427.706,374.907 427.706,374.671 432.475,374.671 432.475,375.006 437.244,375.006 437.244,374.691 442.013,374.691 442.013,374.833 446.782,374.833 446.782,375.058 
 451.551,375.058 451.551,375.301 456.32,375.301 456.32,375.43 461.089,375.43 461.089,375.307 465.858,375.307 465.858,375.2 470.627,375.2 470.627,375.301 
 475.396,375.301 475.396,375.2 480.165,375.2 480.165,375.392 484.934,375.392 484.934,375.594 489.703,375.594 489.703,375.712 494.472,375.712 494.472,375.69 
 499.241,375.69 499.241,375.605 504.01,375.605 504.01,375.63 508.779,375.63 508.779,375.69 513.548,375.69 513.548,375.646 518.317,375.646 518.317,375.698 
 523.087,375.698 523.087,375.723 527.856,375.723 527.856,375.75 532.625,375.75 532.625,375.761 537.394,375.761 537.394,375.764 542.163,375.764 542.163,375.729 
 546.932,375.729 546.932,375.611 551.701,375.611 551.701,375.564 556.47,375.564 556.47,375.6 561.239,375.6 561.239,375.633 566.008,375.633 566.008,375.677 
 570.777,375.677 570

In [ ]:
savefig("spektrum vgl iso.pdf")